The following is the code and explanation for the second assignemnt of NLP. I implemented CASE 1.

## Imports, constants

In [1]:
import time
import nltk
import openai
from Document import Document
import os
import pandas as pd
import sys

print("=== Downloading necessary NLTK pakages, if not already present")

d1 = nltk.download("punkt", quiet=True)
d2 = nltk.download("stopwords", quiet=True)
d3 = nltk.download("wordnet", quiet=True)

# Necessary constants.
_MODEL = "llama-13b-chat"
_CONTEXT_SIZE = 8192  # In Bytes
_DISTANCE_THRESHOLD = 0.20

print(f"=== Loaded\nUsing model `{_MODEL}'")
print(
    f"Context window limit: {_CONTEXT_SIZE} Bytes, i.e {round(_CONTEXT_SIZE / (2**20), 4)} MB"
)
print(f"Distance threshold for the slices: {_DISTANCE_THRESHOLD}")


# The Api Key and the client to be used to query the LLM
# The key is obfuscated, dividend into three, to make crawlers' life miserable after making the repo public
# In a nutshell, It's just the api key written so that it's not a clear string

_a1 = "AbZ".split("b")[1]
_a2 = "BSaNbNTlp0o0bILov9Z3U7XmnP4DhwrV24jgq"
_a3 = "A7kX0SPThAArXd0jNZxQ2WZ"
_build_api = lambda x: (x + _a1)

client = openai.OpenAI(
    api_key=_build_api(f"LL-{_a3}2l1{_a2}"), base_url="https://api.llama-api.com"
)

=== Downloading necessary NLTK pakages, if not already present
=== Loaded
Using model `llama-13b-chat'
Context window limit: 8192 Bytes, i.e 0.0078 MB
Distance threshold for the slices: 0.2


I use four documents as tests. The first two are very small, the third one is larger but still small enough, the fourth one is too big to fit into the window size.
I test the document class and the distance function. The function has been implemented as a _cosine distance_. Check the method for details.

In [2]:
# Some example documents to be used as examples
test_paths = ["./test1.txt", "./test2.txt", "./test3.txt", "./test4.txt"]
# We instantiate our class `Document` that will hold the actual text, the tokens etc
# Internally, it is represented as a Bag Of Word
document1 = Document(path=test_paths[0])
document2 = Document(path=test_paths[1])
document3 = Document(path=test_paths[2])
document4 = Document(path=test_paths[3])

# We print their text
text1 = document1.text(escape=False)
text2 = document2.text(escape=False)
text3 = document3.text(escape=False)
text4 = document4.text(escape=False)

print(f"First document: '{text1}'\n")
print(f"Second document: '{text2}'\n")
print(f"Third document: '{text3}'\n")
print(f"Fourth document: '{text4[:1000]} [CLIPPED]'\n")

# Let's check the distance function (implemented as a cosine distance --- see the function method for details)
# by testing different document pairs
print(
    f"~~~~~~~ Distance examples\nDistance between the first and second document: {document1.distance(document2)}"
)
print(f"Distance between the first and third document: {document1.distance(document3)}")
print(
    f"Distance between the second and third document: {document2.distance(document3)}"
)
print(f"It's symmetric: {document3.distance(document2)}")
print(
    f"The distance between identical documents is zero: {document1.distance(document1)}"
)

First document: 'Dimmi la prima terzina della Commedia di Dante Alighieri .'

Second document: 'Dimmi la seconda terzina del terzo canto della Commedia di Dante Alighieri .'

Third document: 'Give me , in ten simples steps or fewer , a procedure to create an algorithm that implements the following assignment : [ start ] The method is based on the following pipeline : * When the input is below the standard size of the context window ( 128 Mb ) is then passed as it is to the LLM ; * When the input is above the standard size is subdivided in a finite number of slices each of a size that fits the context window and such that they sum to a number N greater than or equal to the size of the input length ; * The criteria to generate a coverage as provided above are : *_* Two slices can overlap ; *_* No slice is included in another one ; *_* When two adjacent slices are settled , the two slices have to be different enough . Ideal solutions will be based on the comparison of two slices based on 

## Algorithm

### Motivation

Consider a document $d$. We can represent it as a sequence of tokens $(t_1,\dotsc, t_n)$, for $n$ indexes, $1$ to $n$. For each document whose size $|d|$ is greater than the context window size $\tau$, the problem consists in finding $m$ indexes $s_1, \dotsc, s_m$ s.t. $s_i < s_j$ for $i < j$, and such that the resulting "slices" (really, documents) $d_1 = (t_1, \dotsc, s_1), d_2 = (s_1 +1, \dotsc, s_2), \dotsc, d_{m+1} = (s_m +1, \dotsc, t_n)$ have all sizes smaller than $\tau$ and such that the distance $D(d_i, d_{i+1})$ for $i = 1,\dotsc,m$ is greater than another given threshold $\ell$. Concretely, in our case we have $\tau$ equal to the LLM (Llama) context window size, and $\ell = 0.2$.

Clearly, this is, in general, a hard problem: $m$ is not given and each slice index can take up to $n$ values.

Instead of an inefficient implementation of this optimization problem (that would require looking through lots of values for $m$ and for each of the slice indexes) I have implemented an efficient slicing method based on a recursive partitioning of the document, similar to how Decision Trees work.

### Description
It starts with dividing the given document exactly in half, if it's actually larger than the context window size (otherwise, it just returns the whole document as a single slice). 
Then, for each slice, it recursively applies the same procedure, splitting them in half. This continues until all the resulting slices are smaller than the context window size. Now, the algorithm computes the distances between all consecutive slices. If some slices are too close to each other, it splits again both (going from 2 slices to 4, that are of course guaranteed to be smaller than the context window size), and the process is repeated from the distance computation point onward, until all slices are distant enough.

This works due to the insight that, in general, smaller slices are more diverse than larger slices. I will empirically validate this insight in a short moment. For now, consider the following example: a document with text "Dì sempre questo: dì sempre questo". Splitting it in half will yeld two slices with distance 0: "Dì sempre questo | dì sempre questo" which cannot be accepted. Therefore, the algorithm would resplit the two slices, obtaining: "Dì sempre | questo | dì sempre | questo" (assuming it rounds up the slicing index). All the consecutive slices have now distance 1 and the slices are accepted.

On average, the slices will be more distant. There is however the problem of convergence: does the algorithm always end by returning slices that are different and small enough? No, if we allow documents to have identical consecutive tokens. This because, in the worst case, each token of the document will correspond to one slice, and if we have two identical consecutive token then it will return slices with distance 0 that cannot be splitted anymore.

My code avoids this problem at the source: when a document is parsed, consecutive tokens are merged together like this: "same same same different" -> "same_same_same different" (check the Document class __init__ method). This guarantee that the procedure will end with slices that are distant enough, because, if we reach the worst case, all slices will have distance 1. 

#### Code

In [3]:
# Helper used in the main algorithm, 'split_document'.
# This recursively half a document into slices that are then slices etc. until
# all slices are smaller than the context window size
def recursive_halve(doc):
    # Return the document if it's small enough. This is the base case
    if doc.size() <= _CONTEXT_SIZE:
        return [doc]
    # Return value. This is a named tuple, i.e. a tuple with fields
    # ["left", "right", "size_left", "size_right", "distance"])
    # Left and right are the two slices, the remaining three fields are the named statistics:
    # The two sizes in bytes and the distance between them
    return_value = doc.split_half()
    # Apply it recursively to the left slice and the right slice
    # The '+' concatenates the two returned array
    return recursive_halve(return_value.left) + recursive_halve(return_value.right)


# Helper method that computes the distances between consecutive slices
def get_distances(slices):
    # Only if there are at least two slices
    if len(slices) > 1:
        distances = []
        for i in range(1, len(slices)):
            # Distance between consecutive slices
            distance = slices[i - 1].distance(slices[i])
            distances.append(distance)
        return distances
    else:
        return []


# -- > MAIN algorithm <--
def split_document(doc):
    # Recursively split the document until all slices are smaller than the context window size
    slices = recursive_halve(doc)
    # Base case: the document was small enough
    if len(slices) == 1:
        return slices
    # Now we have the document evenly split into slices smaller than the context window size.
    # They might be not distant enough
    distant_enough_flag = False
    # Of course, the first turn it always start
    while not distant_enough_flag:
        # Holder for the slices
        temporary = []
        # Assume it's true until you see otherwise. Hence, if we don't set it to False
        # in the following loop, we will do just one round of the algorithm, as it will
        # break the next turn
        distant_enough_flag = True
        i = 0
        while i < len(slices):
            # If we go over the number of slices, we need to append the current slice to temporary
            # otherwise we will never return the last slice.
            # This is needed because we look one index ahead (5 lines down this line)
            if i + 1 >= len(slices):
                temporary.append(slices[i])
            else:
                # If two consecutive slices are not distant enough:
                if slices[i].distance(slices[i + 1]) < _DISTANCE_THRESHOLD:
                    # Reset the flag so we continue the loop
                    distant_enough_flag = False

                    # Then, make two splits. See above for `return value' description
                    return_value_1 = slices[i].split_half()
                    return_value_2 = slices[i + 1].split_half()
                    # Add these 4 slices to temporary
                    temporary.extend(
                        [
                            return_value_1.left,
                            return_value_1.right,
                            return_value_2.left,
                            return_value_2.right,
                        ]
                    )
                    # Increment by one so we skip the next slice (that we have already split, it's slices[i + 1])
                    i += 1
                else:
                    # Just add the slices to temporary.
                    temporary.append(slices[i])
            i += 1
        # The next turn we will work with the found slices
        # If we have found no violation of the distance threshold at the end
        # then distant_enough_flag is True and we break from the loop. If we have found at least
        # a violation, we redo another round
        slices = temporary

    return slices

### Empirical evaluation of average distance, as a function of the number of slices

To validate empirically my insight, I use 200 documents downloaded from Wikipedia and I split them up to 5 times. Every time I split them, I note the number of slices and the average distance between consecutive slices at that number of slices. I do this for all documents, obtaining five statistics for 200 documents. I then average them and print them. The average distance should increase with the number of slices,

In [4]:
# Compute the splits distance statistics
def compute_splits_dist_stats(nsplits=3, ndocs=100):
    # Will contain tuples ('nslices', 'distance_between_slices')
    # Every document will contribute to this data
    data = list()
    # The files on which we'll work. They are contained in the tests folder. We
    # sort them alphabetically, so in case we can debug more easily
    files = sorted(os.listdir("./for_empirical_test/"))
    for path in files[:ndocs]:
        # Load the document
        doc = Document(path=f"./for_empirical_test/{path}")
        # Where the slices are stored. We start with the document itself
        slices = [doc]
        stop_flag = False  # flag to stop after we have reached documents too small
        for _ in range(nsplits):
            if stop_flag:
                break
            # Temporary list to hold the slices for this computation
            temporary = []
            for d in slices:
                # Return value. This is a named tuple, i.e. a tuple with fields
                # ["left", "right", "size_left", "size_right", "distance"])
                # Left and right are the two slices, the remaining three fields are the named statistics:
                # The two sizes in bytes and the distance between them
                return_value = d.split_half()
                # They are too small: set the flag so we exit
                if return_value.left.N() == 0 or return_value.right.N() == 0:
                    stop_flag = True  # So we break the next turn
                # Add to the temporary folder the two slices
                temporary.extend([return_value.left, return_value.right])

            # We have the temporary folder, holding the slices. Compute the distances and then append it to `data'.
            distances = get_distances(temporary)
            nslices = len(temporary)
            for d in distances:
                data.append((nslices, d))
            # Now the docs, used to slice, are the new slices themselves
            slices = temporary

    return pd.DataFrame(data, columns=["nslices", "distance"])

Compute the data, with 200 documents and 6 splits

In [5]:
data = compute_splits_dist_stats(nsplits=6, ndocs=200)

Print the data. Row: number of slices. Column: average distance between consecutive slices, given the `nslices` number of slices.

This show how, by going with more splits (and thus, smaller ones), the average distance between slices increases

In [6]:
print(data.groupby(by="nslices").mean())

         distance
nslices          
2        0.670026
4        0.753973
8        0.814405
16       0.868831
32       0.909452
64       0.940843


## Tests

I show that the algorithm works on the four test documents, by looking at how many slices are extracted, how large they are and what are the distances between slices for each document.

In [7]:
# Remember the test_paths in the second cell
# test_paths = ["./test1.txt", "./test2.txt", "./test3.txt", "./test4.txt", "./test5.txt"]

for path in test_paths:
    print(f"--------------- {path} ---------------")
    doc = Document(path=path)
    print(f"Original document size: {doc.size()} Bytes")
    print(f"Maximum context window size: {_CONTEXT_SIZE} Bytes")
    # Find the slices
    slices = split_document(doc)
    print(f"*{len(slices)}* slices extracted")

    # The sizes of the slices
    sizes = [d.size() for d in slices]
    print(f"Slice sizes: {sizes}")
    max_size = max(sizes)
    print(f"Maximum size among slices: {max_size} Bytes")
    assert max_size <= _CONTEXT_SIZE

    # Now the distances
    distances = get_distances(slices)
    if len(distances) > 0:
        min_distance = min(distances)
        print(f"Distances: {distances}")
        print(f"Minimum distance between consecutive slices: {min_distance}")
        assert min_distance >= _DISTANCE_THRESHOLD
    else:
        print("Because there are fewer than 2 slices, a distance cannot be computed")

--------------- ./test1.txt ---------------
Original document size: 49 Bytes
Maximum context window size: 8192 Bytes
*1* slices extracted
Slice sizes: [49]
Maximum size among slices: 49 Bytes
Because there are fewer than 2 slices, a distance cannot be computed
--------------- ./test2.txt ---------------
Original document size: 64 Bytes
Maximum context window size: 8192 Bytes
*1* slices extracted
Slice sizes: [64]
Maximum size among slices: 64 Bytes
Because there are fewer than 2 slices, a distance cannot be computed
--------------- ./test3.txt ---------------
Original document size: 1059 Bytes
Maximum context window size: 8192 Bytes
*1* slices extracted
Slice sizes: [1059]
Maximum size among slices: 1059 Bytes
Because there are fewer than 2 slices, a distance cannot be computed
--------------- ./test4.txt ---------------
Original document size: 45178 Bytes
Maximum context window size: 8192 Bytes
*8* slices extracted
Slice sizes: [5394, 5453, 5668, 5873, 5692, 5695, 5553, 5850]
Maximum 

### Actual LLM querying

In [8]:
# The function to query the LLM
def query_LLM(doc):
    query_try = 0
    response = None
    # We try up to ten times. (The API could return error)
    while query_try < 10:
        try:
            # We make the request. We use chat completion API: we specify the model (Llama)
            # and append two messages: the first one is our specification for the system,
            # where we state that it must be serious assistant, the second one is our question:
            # it's the passed document/slice text
            response = client.chat.completions.create(
                model=_MODEL,
                messages=[
                    {"role": "system", "content": "You are a serious assistant"},
                    {"role": "user", "content": doc.text()},
                ],
            )
            # If it works, we break from the loop
            break
        except:
            print(response)
            print("Error. Waiting 3 seconds and trying again.")
            time.sleep(3)
            query_try += 1
    if query_try == 10:
        print(f"Reached the limit of {10} tries")
        return ""
    else:
        # The response
        return response.choices[0].message.content

#### Main loop

In [9]:
# Helper method to print answers
def print_answers(answers, clip=True):
    for i, answer in enumerate(answers):
        # If we want to clip and the answer is actually larger the clip size, clip it
        if clip and len(answer) > 1000:
            print(
                f"🤖🤖🤖\n============= Answer {i} =============\n{answer[:1000]}\n\/\ CLIPPED \/\ \n"
            )
        else:
            print(f"🤖🤖🤖\n============= Answer {i} =============\n{answer}\n")


chars = "/—\|"  # Used to print the loading bar

# --> MAIN loop <--
for path in test_paths:
    print(
        f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ {path} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    )
    doc = Document(path=path)
    print(f"Text: ['{doc.text()}']\n")
    # Compute slices
    slices = split_document(doc)
    collated_answers = []  # Where we gather all the answers
    for i, d in enumerate(slices):
        # Print nice loading bar
        sys.stdout.write("\r[" + chars[i % len(chars)] + "] Querying the LLM... ")
        time.sleep(0.1)
        sys.stdout.flush()

        # We query the LLM and append the result to the collated answers
        collated_answers.append(query_LLM(d))
        # Wait a second, just to avoid overloading the API
        time.sleep(1)
    print(f"({len(slices)} slices) \n")
    print_answers(collated_answers, clip=True)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ./test1.txt ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Text: ['Dimmi la prima terzina della Commedia di Dante Alighieri .']

[/] Querying the LLM... (1 slices) 

🤖🤖🤖
============= Answer 0 =============
La prima terzina della Divina Commedia di Dante Alighieri è:

"Nel mezzo del cammin di nostri mortalitare,
che la diritta rinova nel coraggio,
è quan nostro intelletto fa figura
de' begli occhi che si son chiusi e appena
fornite lor veder li miei dolci e lieta
d'ogne mia voglia e d'ogne mia facelta."

(Paradiso, Canto I, versi 1-8)

"Midway upon the journey of our life,
I found myself in a dark forest,
where the straight path had been lost sight of.
How hard it is to say what I saw then!
But my desire and my will were strengthened
to see the mighty virtue of the noble,
and to follow the gentle heart of my lady."

(Paradiso, Canto I, verses 1-8, trans. C.H. Whitcomb)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ./test2.txt ~~~~~~~~~~~~~~~~~~~~~~~~~~